In [1]:
# Setup environment and import standard modules
import sys
import os
import numpy as np

# Import custom modules
sys.path.append(os.path.join(os.getcwd(), 'modules'))
import plottools as pt
import neural_analysis as na

# Autoreload (see https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html)
%load_ext autoreload
%autoreload

Invoking __int__.py for neural_analysis.


In [2]:
# Load data
df = na.proc.load_mgr_data()

In [3]:
# Calculate movement onset
na.proc.calc_movement_onset(df)

In [25]:
# Plot reach trajectories and speed profile
na.plot.reach_trajectories(df)
na.plot.speed_profile(df)

# To-do: Make these in-line figures?

**Prototpying binning**

Here we want to:
* Iterate over each trial
* Bin the trials over some pre-define time grid

The time grid should be the same for each trial, and be determined by the time variable for that trial.

Other considerations:
* We do not know the valid channel/sort combinations beforehand
* The binning function should take a list of channel/sort combinations and return an array of binned spike times for each channel/sort.
* If the channel/sort is not found, the corresponding row in the binned data should still exist (i.e., be zeros)

In [39]:
# Specify parameters
bin_width = 20
bin_start = 0
kin_offset = 150
valid_sort = range(1, 31)

# Find the list of valid sort/channel combinations.  Can do this using a set.
all_units = set()
for index, trial in df.iterrows():
   all_units.update(
       [(c, s) for c, s in zip(trial.spike_channel, trial.spike_sort)
            if s in valid_sort
       ]
   )

all_units = sorted(all_units)  # Note -- converts set to a list
n_units = len(all_units)
print('Number of units: {}'.format(n_units))
print(all_units)

Number of units: 142
[(1, 1), (2, 1), (3, 1), (3, 2), (4, 1), (5, 1), (6, 1), (6, 2), (7, 1), (7, 2), (8, 1), (8, 2), (9, 1), (10, 1), (12, 1), (12, 2), (13, 1), (13, 2), (14, 1), (15, 1), (15, 2), (16, 1), (16, 2), (18, 1), (19, 1), (19, 2), (20, 1), (20, 2), (21, 1), (22, 1), (23, 1), (23, 2), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (28, 2), (29, 1), (30, 1), (30, 2), (31, 1), (31, 2), (32, 1), (32, 2), (34, 1), (35, 1), (35, 2), (36, 1), (37, 1), (37, 2), (38, 1), (38, 2), (40, 1), (40, 2), (41, 1), (41, 2), (42, 1), (43, 1), (43, 2), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (48, 2), (49, 1), (50, 1), (51, 1), (52, 1), (52, 2), (53, 1), (53, 2), (54, 1), (54, 2), (55, 1), (55, 2), (56, 1), (57, 1), (57, 2), (58, 1), (58, 2), (59, 1), (60, 1), (61, 1), (62, 1), (62, 2), (63, 1), (63, 2), (64, 1), (65, 1), (65, 2), (66, 1), (66, 2), (66, 3), (67, 1), (67, 2), (68, 1), (68, 2), (68, 3), (69, 1), (70, 1), (70, 2), (71, 1), (72, 1), (74, 1), (75, 1), (75, 2), (76, 1), (76, 2), (

In [82]:
def bin_data(x, t, t_onset, t_offset, mode='mean'):
    """
    Bin data.
    
    Inputs:
    :x
    :t
    :t_bin
    
    Note: this function expects column vector convention, where rows are
    dimensions, and columns are observations (in this case time).
    """
    # Input is a 1D array, expand to 2D
    if x.ndim == 1:        
        x = np.expand_dims(x, 0)
    
    # Initialize binned data matrix
    n_dim = x.shape[0]
    n_bins = len(t_onset)
    x_bin = np.zeros([n_dim, n_bins])
    
    # Iterate over bins
    for i in range(n_bins):
        # Determine time mask
        mask = (t >= t_onset[i]) & (t < t_offset[i])
        
        # Bin data depending on the mode
        if mode is 'mean':
            x_bin[:, i] = x[:, mask].mean(1)
        elif mode is 'sum':
            x_bin[:, i] = x[:, mask].sum(1)
    
    return x_bin


In [84]:
# Get data from a single trial for debugging purposes
trial = df.iloc[0]

# Get time for the current trial and define the bin times
t = trial.time
onset_idx = trial.onset_idx
t = t - t[onset_idx]  # Correct for movement onset

# Define bin times.  The '+1' here allows for the end of the last bin to
# occur at the last time point.
t_bin = np.arange(bin_start, t[-1] + 1, bin_width)
t_onset = t_bin[0:-1]
t_offset = t_bin[1:]

# Initialize spike count matrix
n_bins = len(t_onset)
X = np.zeros([n_units, n_bins])

# Iterate over all channel/sort combinations and bin the spike times
for ch, srt, st in zip(trial.spike_channel, trial.spike_sort, trial.spike_times):
    # Check to make sure channel/sort is in the list of valid units
    if (ch, srt) not in all_units or st is None:
        continue
    
    # In order to bin the spike data, first convert it into a raster with ms
    # resolution.  To do this, the spike times need to be converted into an index.
    s_ind = st - np.min(st)  # Now s_ind[0] is the first spike
    raster = np.zeros(np.max(s_ind) + 1)
    raster[s_ind] = 1
    t_raster = np.arange(len(raster)) + np.min(st) - trial.time[onset_idx]

    # Bin spike times
    binned_spikes = bin_data(raster, t_raster, t_onset, t_offset, 'sum')
    
    # Add spikes to matrix
    mask = np.array([list(v) == [ch, srt] for v in all_units])
    X[mask, :] = binned_spikes

# Bin kinematic data (all dimensions)
K_bin = bin_data(trial.pos, trial.time - trial.time[trial.onset_idx], t_onset, t_offset, 'mean')
print(trial.pos)
print(K)

# Next step -- plot this to make sure that the binning function is working as expected

[[  10.9495025    11.02827758   11.10702512 ...   17.80126249
    17.800959     17.80089692]
 [ -15.64970687  -15.69589775  -15.72446795 ...  138.7860194
   138.78644389  138.78653764]
 [-103.72519957 -103.98907834 -104.20955602 ... -133.57320145
  -133.5735953  -133.57367789]]
[[  14.72791857   30.28043224   38.11672064   36.31756417   28.1754685
    22.65002347   19.32573762]
 [  -1.74477179   49.11989172   91.3173664   116.8075731   131.17361906
   136.18113486  137.97336785]
 [-105.28905283 -102.93420178 -109.31785784 -118.82705957 -127.45903682
  -130.53653906 -132.30387222]]


In [107]:
np.array(all_units)
np.isin(all_units, [ch, srt]).all(1)
all_units[0]

(1, 1)

In [46]:
# Test binning

# Define binning parameters
bin_width = 100
bin_start = 0
kin_offset = 150
valid_sort = range(1, 31)

# Get data from a single trial for debugging purposes
trial = df.iloc[0]

# Get time for the current trial and define the bin times
t = trial.time
onset_idx = trial.onset_idx
t = t - t[onset_idx]  # Correct for movement onset

# Define bin times.  The '+1' here allows for the end of the last bin to
# occur at the last time point.
t_bin = np.arange(bin_start, t[-1] + 1, bin_width)
t_onset = t_bin[0:-1]
t_offset = t_bin[1:]

# Get spike data
ch = trial.spike_channel[1]
srt = trial.spike_sort[1]
st = trial.spike_times[1]

[f is (1, 1) for f in all_units]
all_units[0]

(1, 1)

In [66]:
# Get spike data
ch = trial.spike_channel[1]
srt = trial.spike_sort[1]
st = trial.spike_times[1]

np.array([list(v) == [ch, srt] for v in all_units])

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,